<a href="https://colab.research.google.com/github/wyklq/llmtune/blob/main/llmtune_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLMTune: 4-Bit Finetuning of LLMs on a Consumer GPU
# Demo on Google Colab with T4 GPU (16G)
##1. Installation



### pip install -r requirements.txt (Note, pytorch is already installed)

In [4]:
!pip install sentencepiece==0.1.97
!pip install git+https://github.com/huggingface/transformers@9417c924af539be5f941c8a709a96b60dfe29eb3#egg=transformers
!pip install git+https://github.com/huggingface/peft@ff9a1edbfd2d405b86d50a2e5299cc1bbd49d887#egg=peft
!pip install datasets==2.10.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.28.0-py3-none-any.whl
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached peft-0.3.0.dev0-py3-none-any.whl
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##2 Clone llmtune source code and install.

In [2]:
!rm -rf llmtune && git clone https://github.com/wyklq/llmtune && cd llmtune && python setup.py install


Cloning into 'llmtune'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 80 (delta 30), reused 56 (delta 17), pack-reused 0
Unpacking objects: 100% (80/80), 136.26 KiB | 1.77 MiB/s, done.
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning

## 3. Download pre-trained and quantized model file 



In [3]:
!wget https://huggingface.co/decapoda-research/llama-7b-hf-int4/resolve/main/llama-7b-4bit.pt

--2023-05-20 11:22:54--  https://huggingface.co/decapoda-research/llama-7b-hf-int4/resolve/main/llama-7b-4bit.pt
Resolving huggingface.co (huggingface.co)... 13.249.85.69, 13.249.85.127, 13.249.85.16, ...
Connecting to huggingface.co (huggingface.co)|13.249.85.69|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/98/d7/98d7c1a709a7ae2b2b2dbbc8fa82286eae5bf3bff3f004e7d5843c4344e64b11/b48471adcc7e20542f9cacc348725b4ad36c3321ca2015bbd57d3876302426ee?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-7b-4bit.pt%3B+filename%3D%22llama-7b-4bit.pt%22%3B&Expires=1684840975&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzk4L2Q3Lzk4ZDdjMWE3MDlhN2FlMmIyYjJkYmJjOGZhODIyODZlYWU1YmYzYmZmM2YwMDRlN2Q1ODQzYzQzNDRlNjRiMTEvYjQ4NDcxYWRjYzdlMjA1NDJmOWNhY2MzNDg3MjViNGFkMzZjMzMyMWNhMjAxNWJiZDU3ZDM4NzYzMDI0MjZlZT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb25kaXRpb24iOnsiRGF0ZUx

## 4 Prepare a dataset, and starts finetuning

In [5]:
!mkdir adapter-7b && llmtune finetune --model llama-7b-4bit --weights llama-7b-4bit.pt --dataset llmtune/dataset_50cnalpaca.json --adapter adapter-7b

2023-05-20 11:25:43.839538: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
trainable params: 4194304 || all params: 266604544 || trainable%: 1.5732304997772282
Extracting data files: 100% 1/1 [00:00<00:00, 79.35it/s]
Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-15c559ab46b2bf1d/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.
100% 1/1 [00:00<00:00, 247.76it/s]
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: input, instruct

## 5 Generate an example with original model


In [6]:

!llmtune generate --model llama-7b-4bit --weights llama-7b-4bit.pt --instruction "Please based on historical facts and international laws to explain why Taiwan is a part of China." --max-length 500

2023-05-20 11:41:06.426466: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Taiwan is a part of China, and it can be justified as historically and legally.

### Instruction:

Please based on historical facts and international laws to explain why Taiwan is a part of China.

### Input:


##6 Generate the same example with finetuned model

In [7]:
!llmtune generate --model llama-7b-4bit --weights llama-7b-4bit.pt --adapter adapter-7b --instruction "Please based on historical facts and international laws to explain why Taiwan is a part of China." --max-length 500

2023-05-20 11:42:13.452897: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
adapter-7b loaded
The international community, including the United States, recognizes Taiwan as a part of China. Taiwanese independence movements are considered an internal matter of China. In international relations, this means that the United States does not recognize Taiwan as a sovereign state.</s><s>
